<a id=0></a>
# 4.Categorical Plot

---
### [1.SearbornのデータセットからDataFrameを作成 ](#1)
### [2.欠損値の処理 ](#2)
### [3.欠損した年齢を考えてみる（プロットの例を示す）](#3)
### [4.カテゴリカルプロット](#4)
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

---
<a id=1></a>
[Topへ](#0)

---
## 1. SearbornのデータセットからDataFrameを作成

* データセット(Titanic)を読み込む
* データセットの理解と整理、Kaggleの紹介
* 重複カラムの処理
---

データセット(Titanic)を読み込む

In [2]:
sns.get_dataset_names()

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'dowjones',
 'exercise',
 'flights',
 'fmri',
 'geyser',
 'glue',
 'healthexp',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'seaice',
 'taxis',
 'tips',
 'titanic']

データセットの理解と整理、Kaggleの紹介

<p>
    <a href="https://www.kaggle.com/competitions/titanic/data" target="_blank" rel="noreferrer noopener">
        Kaggle Dataset Titanic のページへ
    </a>
</p>

    survival : Survival	0 = No, 1 = Yes  
    pclass   : Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd  
    sex	  : Sex	  
    age      : Age in years  	
    sibsp    : # of siblings / spouses aboard the Titanic  (兄弟姉妹 / 配偶者)  
    parch    : # of parents / children aboard the Titanic	(親 / 子)  
    fare     : Passenger fare	
    embarked : Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton  
    who      : man or female or child
    class, who, adult_male, deck, embark_town, alive, alone
    ※ 一部変更しています

In [3]:
df = sns.load_dataset(name="titanic")

重複カラムの処理（'alive', 'class', 'adult_male', 'embark_town', 'alone'）

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [6]:
df["survived"].unique(), df["alive"].unique()

(array([0, 1], dtype=int64), array(['no', 'yes'], dtype=object))

In [7]:
df[["survived", "alive"]].value_counts()

survived  alive
0         no       549
1         yes      342
dtype: int64

In [8]:
df[["pclass", "class"]].value_counts()

pclass  class 
3       Third     491
1       First     216
2       Second    184
dtype: int64

In [10]:
df[["embarked", "embark_town"]].value_counts()

embarked  embark_town
S         Southampton    644
C         Cherbourg      168
Q         Queenstown      77
dtype: int64

In [11]:
df[["who", "adult_male"]].value_counts()

who    adult_male
man    True          537
woman  False         271
child  False          83
dtype: int64

In [15]:
df[(df["sibsp"] == 0) & (df["parch"] == 0)]["alone"].unique()

array([ True])

In [16]:
df[(df["sibsp"] != 0) | (df["parch"] != 0)]["alone"].unique()


array([False])

<p>
    'alive', 'class', 'adult_male', 'embark_town', 'alone' のカラムを削除 
    </p>

In [21]:
df.drop(columns=["alive", "class", "adult_male", "embark_town", "alone"], inplace=True)

KeyError: "['alive', 'class', 'adult_male', 'embark_town', 'alone'] not found in axis"

In [23]:
df.shape

(891, 10)

---
<a id=2></a>
[Topへ](#0)

---
## 2. 欠損値の処理

* 欠損値の確認
* 欠損値の処理('deck')
* 欠損値の処理('embarked')：予測を試みる
---

欠損値の確認

In [26]:
df.isnull().sum(axis=0)

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
who           0
deck        688
dtype: int64

欠損値の処理('deck')

In [28]:
df["deck"].value_counts().sort_index()

A    15
B    47
C    59
D    33
E    32
F    13
G     4
Name: deck, dtype: int64

In [31]:
df.groupby("deck")["survived"].describe()

,count,mean,std,min,25%,50%,75%,max
deck,,,,,,,,
A,15.0,0.466667,0.516398,0.0,0.00,0.0,1.0,1.0
B,47.0,0.744681,0.440755,0.0,0.50,1.0,1.0,1.0
C,59.0,0.593220,0.495450,0.0,0.00,1.0,1.0,1.0
D,33.0,0.757576,0.435194,0.0,1.00,1.0,1.0,1.0
E,32.0,0.750000,0.439941,0.0,0.75,1.0,1.0,1.0
F,13.0,0.615385,0.506370,0.0,0.00,1.0,1.0,1.0
G,4.0,0.500000,0.577350,0.0,0.00,0.5,1.0,1.0


欠損値の処理('embarked')：予測を試みる

---
<a id=3></a>
[Topへ](#0)

---
## 3. 欠損した年齢を考えてみる（プロットの例を示す）

* 年齢を確認
* 年齢の分布
    * **sns.histplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.histplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.histplot のページへ
        </a>
    </p>


* 'age'と'pclass'の関係
* 'age'と'sex'の関係
* 相関関係（correlation）
    * **sns.heatmap()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.heatmap.html" target="_blank" rel="noreferrer noopener">
            Seaborn.heatmap のページへ
        </a>
    </p>


* 個別の予測は困難であり、Seabornの学習がメインのため、レコードをすべて削除する
* CSVファイルとして保存する

---

年齢を確認

年齢の分布

In [ ]:
# google colaboratoryの場合
# !pip install japanize-matplotlib

In [ ]:
# google colaboratoryの場合
# import japanize_matplotlib
# japanize_matplotlib.japanize()

'age'と'pclass'の関係

'age'と'sex'の関係

相関関係

個別の予測は困難であり、Seabornの学習がメインのため、レコードをすべて削除する

CSVファイルとして保存する

---
<a id=4></a>
[Topへ](#0)

---
## 4. カテゴリカルプロット

* キャットプロットとストリッププロット
 * **sns.catplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.catplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.catplot のページへ
        </a>
    </p>
    

* カウントプロット
    * **sns.countplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.countplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.countplot のページへ
        </a>
    </p>


* バープロット
 * **sns.barplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.barplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.barplot のページへ
        </a>
    </p>


* ボックスプロット
 * **sns.boxplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.boxplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.boxplot のページへ
        </a>
    </p>
    

* バイオリンプロット
 * **sns.violinplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.violinplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.violinplot のページへ
        </a>
    </p>
    
    
* スウォームプロット
 * **sns.swarmplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.swarmplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.swarmplot のページへ
        </a>
    </p>
    


---

キャットプロットとストリッププロット

カウントプロット

バープロット

ボックスプロット

バイオリンプロット

スウォームプロット

---
 <a id=4></a>
[Topへ](#0)

---
## 以上
    
---